# 1. 加载数据

In [19]:
# 原始语料，3个文本
strs_train =[
'God is love',
'OpenGL on the GPU is fast',
'Doctor David is PHD']

# 2. 提取Bags of words特征

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(strs_train)
X_train_counts.shape

(3, 11)

In [17]:
X_train_counts.todense()#转换成数值特征Bags of words

matrix([[0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0],
        [0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1],
        [1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0]], dtype=int64)

In [18]:
count_vect.vocabulary_#特征名

{'david': 0,
 'doctor': 1,
 'fast': 2,
 'god': 3,
 'gpu': 4,
 'is': 5,
 'love': 6,
 'on': 7,
 'opengl': 8,
 'phd': 9,
 'the': 10}

# 3. 提取TF特征

In [20]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
# 先提取 Bags of words特征
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(strs_train)
# 再基于Bags of words特征，变换为TF特征
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

(3, 11)

In [21]:
X_train_tf.todense()# 查看TF特征

matrix([[ 0.        ,  0.        ,  0.        ,  0.57735027,  0.        ,
          0.57735027,  0.57735027,  0.        ,  0.        ,  0.        ,
          0.        ],
        [ 0.        ,  0.        ,  0.40824829,  0.        ,  0.40824829,
          0.40824829,  0.        ,  0.40824829,  0.40824829,  0.        ,
          0.40824829],
        [ 0.5       ,  0.5       ,  0.        ,  0.        ,  0.        ,
          0.5       ,  0.        ,  0.        ,  0.        ,  0.5       ,
          0.        ]])

**sklearn的TF计算并不是用 该词出现次数/文档词数，而是直接将Bags of words做Normalization**

# 4. 提取TF-IDF特征

In [38]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
# 先提取 Bags of words特征
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(strs_train)
# 再基于Bags of words特征，变换为TF-IDF特征
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.todense()


matrix([[ 0.        ,  0.        ,  0.        ,  0.65249088,  0.        ,
          0.38537163,  0.65249088,  0.        ,  0.        ,  0.        ,
          0.        ],
        [ 0.        ,  0.        ,  0.43238509,  0.        ,  0.43238509,
          0.2553736 ,  0.        ,  0.43238509,  0.43238509,  0.        ,
          0.43238509],
        [ 0.54645401,  0.54645401,  0.        ,  0.        ,  0.        ,
          0.32274454,  0.        ,  0.        ,  0.        ,  0.54645401,
          0.        ]])

# 5. FeatureHasher

In [48]:
from sklearn.feature_extraction import FeatureHasher

# 将原始数据Hash到10维Feature
h = FeatureHasher(n_features=10, non_negative=True)

# 将数据组成D格式
D=[]
for s in strs_train:
    tmp = {}
    for w in s.split(' '):
        tmp[w] = 1
    D.append(tmp)

f = h.transform(D)
f.toarray()


array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  1.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  1.,  1.,  0.,  1.]])

In [47]:
D

[{'God': 1, 'is': 1, 'love': 1},
 {'GPU': 1, 'OpenGL': 1, 'fast': 1, 'is': 1, 'on': 1, 'the': 1},
 {'David': 1, 'Doctor': 1, 'PHD': 1, 'is': 1}]

# Test

In [30]:
import numpy as np
import scipy.sparse as sp

X = X_train_counts.data

In [31]:
hasattr(X, 'dtype')

True

In [35]:
    if sp.isspmatrix_csr(X):
        return bincount(X.indices, minlength=X.shape[1])
    else:
        return np.diff(sp.csc_matrix(X, copy=False).indptr)


In [37]:
np.log(X.data, X.data)

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])